In [1]:
!pip install itables 

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None


In [15]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import pandas as pd
import itables.options as opt
opt.columnDefs = [{"className":"dt-left", "targets": "_all"}]
opt.lengthMenu = [50, 100, 200, 500]
import urllib, json, requests
import ipywidgets as widgets


<IPython.core.display.Javascript object>

In [16]:
TIME_API = "http://time-api.loci.cat/api/v1"

In [19]:
#visualise LGA 
import json
import requests
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = TIME_API  + "/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    return res

# Loc-I Change Over Time Demo

This notebook demonstrates the prototype Loc-I change over time API available at http://time-api.loci.cat

Some basic API functionality is introduced and a selection of use cases will be demonstrated below.

## Loc-I Time API Basics



### Search for a feature by name/label/code

Example: Find features with label of "Fraser"

In [6]:
w = widgets.Text(
    value='Fraser',
    placeholder='Fraser',
    description='Input label:',
    disabled=False
)
display(w)

Text(value='Fraser', description='Input label:', placeholder='Fraser')

In [9]:
API_ENDPOINT = "/location/find-by-label"

url = TIME_API + API_ENDPOINT
payload = {
    'query': w.value
}
r = requests.get(url, params=payload)
jsondata = r.json()
#jsondata
df = pd.DataFrame (jsondata, columns = ['uri', 'match'])
df

uri,match


### Get a Loc-I Feature at a point in time

Example: Find the CED Commonwealth Electoral District (CED) of Fraser at a point in time, 7th September 2017.

In [50]:
FRASER_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214"
TIME_VARIABLE = "2017-09-07"

In [51]:
API_ENDPOINT = "/temporal/feature"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FRASER_CED,
    'time' : TIME_VARIABLE
}
r = requests.get(url, params=payload)
jsondata = r.json()
jsondata

{'message': 'Internal Server Error', 'code': 500}

In [52]:
mydict = {k: [v] for k, v in jsondata.items()}
df = pd.DataFrame (mydict, columns = mydict.keys())
df

message,code


In [49]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker
from shapely.geometry import shape
fraser_geojson_data = get_geom(FRASER_CED)
g= shape(fraser_geojson_data)

x_coord = g.centroid.coords[0][1]
y_coord = g.centroid.coords[0][0]

map = ipy.Map(center=[x_coord, y_coord], zoom=11)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=fraser_geojson_data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map.add_layer(geo_json1)
map

AttributeError: 'NoneType' object has no attribute 'lower'

## Use case 1.A. Features intersecting at the same time (Fix at a point in time for source feature)

For entity A, in data set X, return identifiers for entities of class 2 from data set Y, that intersect A at time α

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Fraser (ACT), return identifiers for the target features, SA2 regions from the ASGS edition 2 (2016), that intersected the source CED, on 7 September 2017.

In [53]:
FRASER_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214"
TIME_VARIABLE = "2017-09-07"
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#StatisticalAreaLevel2"

In [54]:
API_ENDPOINT = "/temporal/intersects"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FRASER_CED,
    'time' : TIME_VARIABLE,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
jsondata = r.json()
mylist = [[k] for k in jsondata]

In [55]:
df = pd.DataFrame (mylist, columns = ['uri'])
df

uri


In [43]:
from tqdm.notebook import tqdm  as tqdm


In [47]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=11)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

source_geojson = GeoJSON(data=fraser_geojson_data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
map2.add_layer(source_geojson)

with tqdm(total=len(jsondata)) as pbar:
    for uri in jsondata:
        #print(uri)
        geojson_data = get_geom(uri)
        geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity': 0.1, 'weight':1.9, 'fillOpacity':0.2})
        map2.add_layer(geo_json1)
        pbar.update(1)
map2

  0%|          | 0/54 [00:00<?, ?it/s]

Map(center=[-37.7467757320652, 144.8069843868863], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## Use case 1.B. Features intersecting at the same time (Fix at a point in time for both)

For entity A, in data set X, return identifiers for entities of class 2 from data set Y at time b, that intersect A at time α, where time α and b are the same

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Fraser (ACT), return identifiers for the target features, LGA regions, that existed at 2017-09-07, that intersected the source CED, on 7 September 2017.

In [38]:
FRASER_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214"
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
TIME_SOURCE_FEATURE = "2017-09-07"
TIME_TARGET_FEATURE_TYPE = "2017-09-07"

In [39]:
API_ENDPOINT = "/temporal/intersects2"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FRASER_CED,
    'fromTime' : TIME_SOURCE_FEATURE,
    'toTime' : TIME_TARGET_FEATURE_TYPE,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
print(r.url)
jsondata = r.json()
mylist = [[k] for k in jsondata]
df = pd.DataFrame (mylist, columns = ['uri'])
df

http://time-api.loci.cat/api/v1/temporal/intersects2?uri=https%3A%2F%2Flinked.data.gov.au%2Fdataset%2Fasgs%2Fcommonwealthelectoraldivision%2F214&fromTime=2017-09-07&toTime=2017-09-07&toFeatureType=https%3A%2F%2Flinked.data.gov.au%2Fdef%2Fasgs%23LocalGovernmentArea


uri


In [ ]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=11)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

with tqdm(total=len(jsondata)) as pbar:
    for uri in jsondata:
        #print(uri)
        geojson_data = get_geom(uri)
        geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'green', 'opacity': 0.1, 'weight':1.9, 'fillOpacity':0.2})
        map3.add_layer(geo_json1)
        pbar.update(1)
map3

## Use case 1.B. Features intersecting at different times (Specify time points for source feature and target feature types)

For entity A, in data set X, return identifiers for entities of class 2 from data set Y at time b, that intersect A at time α, where time α and b are different

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Fraser (ACT), return identifiers for the target features, LGA regions, that existed at 2020-09-07, that intersected the source CED, on 7 September 2017.

In [40]:
FRASER_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214"
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
TIME_SOURCE_FEATURE = "2017-09-07"
TIME_TARGET_FEATURE_TYPE = "2020-09-07"

In [41]:
API_ENDPOINT = "/temporal/intersects2"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FRASER_CED,
    'fromTime' : TIME_SOURCE_FEATURE,
    'toTime' : TIME_TARGET_FEATURE_TYPE,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
#print(r.url)
jsondata = r.json()
mylist = [[k] for k in jsondata]
df = pd.DataFrame (mylist, columns = ['uri'])
df

uri


## Use case 2.A. Features intersecting input feature between time A and B.

For entity A, in data set X, return identifiers for entities of class 2 from data set Y, that intersected entity A between times α and β

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Fraser (ACT), return identifiers for the target features, ASGS Non-ABS LGAs, that intersected the source CED, on 7 September 2017 and 7 September 2020

In [42]:
FRASER_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214"
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
TIMESPAN_START = "2017-09-07"
TIMESPAN_END = "2020-09-07"

In [43]:
API_ENDPOINT = "/temporal/intersects3"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FRASER_CED,
    'fromTime' : TIMESPAN_START,
    'toTime' : TIMESPAN_END,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
#print(r.url)
jsondata = r.json()
mydict = {k: sorted(v) for k, v in jsondata.items()}
df = pd.DataFrame (mydict, columns = mydict.keys())
df

http://time-api.loci.cat/api/v1/temporal/intersects3?uri=https%3A%2F%2Flinked.data.gov.au%2Fdataset%2Fasgs%2Fcommonwealthelectoraldivision%2F214&fromTime=2017-09-07&toTime=2020-09-07&toFeatureType=https%3A%2F%2Flinked.data.gov.au%2Fdef%2Fasgs%23LocalGovernmentArea


https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214,https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/214/2018-07


In [28]:
import ipycytoscape

In [16]:
import json
with open("DAGData.json") as fi:
    json_file = json.load(fi)

In [17]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(json_file)

In [18]:
cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e'
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])

In [19]:
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLengthVal': 10}, cytoscape_style=[{…